# 🏋🏽‍♂️ Vizualizace během trénování
Vizualizace nám mohou pomoci lépe pochopit proces trénování. V přednášce jsme si říkali, že lze vizualizovat výsledný model pro různé množiny hyperparametrů nebo třeba vizualizovat nějakou metriku jako časovou řadu napříč iteracemi. V tomto notebooku si ukážeme, jak to udělat v praxi.

In [ ]:
# models
from sklearn.tree import DecisionTreeClassifier 

# data generation
from sklearn.datasets import make_blobs
import numpy as np

# plotting
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions 

# interactive elements
from ipywidgets import interact

## 📊 Vizualizace modelu pro různé hodnoty hyperparametrů
Postup si ukážeme na rozhodovacím stromu a jeho hyperparametru `max_depth`.

Začneme tím, že si vygenerujeme nějaká data (např. pomocí funkce [`make_blobs()`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html), která defaultně generuje 2 příznaky) pro klasifikační problém a vytvoříme funkci, která bude vizualizovat natrénovaný model.

In [ ]:
X, y = make_blobs(n_samples=300, centers=4, random_state=0, cluster_std=1.2)

fig, ax = plt.subplots()
ax.scatter(X[:, 0], X[:, 1], c=y, s=50)

In [ ]:
def show_decision_region(X, y, clf):
    fig, ax = plt.subplots(figsize=(16,8))
    
    # styling
    scatter_kwargs = {'edgecolor': None, 'alpha': 0.7}
    contourf_kwargs = {'alpha': 0.3}
    
    # plotting
    plot_decision_regions(X, y, clf=clf, ax=ax, legend=2, scatter_kwargs=scatter_kwargs, contourf_kwargs=contourf_kwargs)

Pojďme se podívat, co nám funkce `show_decision_region` vykreslí 👀.

In [ ]:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X, y)

show_decision_region(X, y, clf)

Nyní vytvoříme funkci, která vykreslí interaktivní vizualizaci. K tomu využijeme funkci `interact`.

👨🏽‍💻 [user guide](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html)

Funkce `interact` funguje následovně – parametrem prodáme funkci, která má nějaké vlastní parametry a `interact` vytvoří UI elementy, které nám umožní tyto parametry nastavit. Po nastavení parametru `interact` je zavolána funkce s vybranými hodnotami a její výstup se vypíše/vykreslí do výstupu buňky.

In [ ]:
def plot_tree_interactive(X, y, depth_min, depth_max):   
    # trains classifier with provided depth and displays decision region
    def interactive_tree(depth):
        clf = DecisionTreeClassifier(max_depth=depth, random_state=0)
        clf.fit(X, y)
        show_decision_region(X, y, clf)

    # interact witdget (calls interactive_tree function with current depth specified by slider value)
    # slider values go from depth_min to depth_max
    return interact(interactive_tree, depth=(depth_min,depth_max))

In [ ]:
_ = plot_tree_interactive(X, y, 1, 8)

Díky vizualizaci můžeme konstatovat, že ideální hloubka je v tomto případě 3 nebo 4. Stromy s menší hloubkou neuměly predikovat všechny třídy a stromy s větší hloubkou byly přeučené.

## 📈 Vizualizace metriky napříč iteracemi
Při výběru výsledného modelu se rozhodujeme podle nějaké metriky (nebo více metrik). Při klasifikaci je to často klasifikační přesnost (angl. classification accuracy).

Predispozicí k tomuto typu vizualizace je, že jsme si během trénování v každé iteraci ukládali hodnotu dané metriky.

Vytvořme si tedy data, která takovou situaci simulují. Představme si, že jsme trénovali rozhodovací strom 🌳 a zkusili jsme ladit hyperparametr `max_depth` pro hodnoty 1 až 11. V každé iteraci jsme si uložili trénovací a validační přesnost.

In [ ]:
train_acc = [0.4880, 0.7023, 0.9047, 0.9226, 0.9404, 0.9642, 0.9642, 0.9821, 0.9940, 0.9940]
val_acc = [0.4210, 0.7192, 0.9298, 0.9122, 0.8771, 0.8771, 0.8771, 0.8421, 0.8421, 0.8771]

depths = range(1,11)

In [ ]:
# styling
blue = '#8592dc'
violet = '#9047A0'
red = '#d14081'

plt.rcParams.update({"axes.grid" : True})
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=[blue, violet, red])
plt.style.use('seaborn-darkgrid')

Graf pak vytvoříme tak, že vykreslíme dva čárové grafy (angl. line charts), jeden pro trénovací a druhý pro validační přesnost.

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
ax.set_xlabel('max depth')
ax.set_ylabel('accuracy')

ax.plot(depths, train_acc,'o-', label='train')
ax.plot(depths, val_acc,'o-', label='validation')

_ = ax.legend()

Z grafu vidíme, že i když trénovací přesnost stále rostla, validační začala od hloubky 3 klesat. To signalizuje přeučení. Nejlepší byl tedy model s hloubkou 3.

### 🛠 Ladění více hyperparametrů
Většinou ladíme více než jeden hyperparametr. V takovém případě můžeme jemně ohnout graf z předchozí sekce (to si ukážeme v notebooku `example.ipynb`). Druhá možnost je například použít graf paralelních souřadnic. Ten v `matplotlib`u neexistuje a museli bychom jej naprogramovat ručně. Naštěstí můžeme použít balíček `plotly`, který tento graf obsahuje.

Navíc je tento graf defaultně interaktivní. Můžete zkusit přeuspořádat osy tím, že kliknete na jejich název a potáhnete je. Také můžete filtrovat, jaký interval hodnot vás na dané ose zajímá. Stačí kliknout na maximální hodnotu a potáhnout myš směrem dolů na minimální hodnotu (nebo naopak).

🗂 [dokumentace](https://plotly.com/python-api-reference/generated/plotly.express.parallel_coordinates.html)

👨🏽‍💻 [examples](https://plotly.com/python/parallel-coordinates-plot/)

In [ ]:
import plotly.express as px
import pandas as pd

rng = np.random.RandomState(1)

d = {
    'max_depth': [5, 5, 5, 8, 8, 8, 10, 10, 10, 5, 5, 5, 8, 8, 8, 10, 10, 10, 5, 5, 5, 8, 8, 8, 10, 10, 10], 
    'min_samples_leaf': [1, 3, 5, 1, 3, 5, 1, 3, 5, 1, 3, 5, 1, 3, 5, 1, 3, 5, 1, 3, 5, 1, 3, 5, 1, 3, 5],
    'min_samples_split': [ 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8],
    'accuracy': rng.rand(27)
}

df = pd.DataFrame(data=d)

fig = px.parallel_coordinates(
    df, 
    color='accuracy', 
    # sets color scale to red-yellow-green
    color_continuous_scale=px.colors.diverging.RdYlGn,
    # sets middle value (accuracy range is 0 to 1, middle is in 0.5)
    color_continuous_midpoint=0.5
)

fig.show()

# 🎉 A to je k trénování vše! 🎉 